<a href="https://colab.research.google.com/github/fidan-c/human-value-detection/blob/main/Value_Det2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import numpy as np


In [ ]:
random.seed(64)
np.random.seed(64)

In [ ]:
# using Conc + Premise as one representation and Stance added to the labels: roberta-base </s> added when concatenating the two columns to one column, lowercased.
!pip install datasets evaluate transformers[sentencepiece]
!pip install transformers
!pip install tokenizer
!pip install simpletransformers

In [ ]:

import tensorflow as tf
from transformers import AutoTokenizer,TFAutoModel, DataCollatorWithPadding
import pandas as pd


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from simpletransformers.classification import (
    MultiLabelClassificationModel, MultiLabelClassificationArgs
)

In [ ]:
# load data - not including chinese
# sep for tsv
arguments_training_no_label = pd.read_csv('/content/drive/MyDrive/transformers/arguments-training.tsv', sep='\t') 
arguments_validation_no_label = pd.read_csv('/content/drive/MyDrive/transformers/arguments-validation.tsv', sep='\t')
arguments_test_data = pd.read_table('/content/drive/MyDrive/transformers/arguments-test.tsv')
arguments_test_data_nahjalbalagha= pd.read_table('/content/drive/MyDrive/transformers/arguments-test-nahjalbalagha.tsv')

labels_training = pd.read_csv('/content/drive/MyDrive/transformers/labels-training.tsv', sep='\t').drop('Argument ID', axis=1)
labels_validation= pd.read_csv('/content/drive/MyDrive/transformers/labels-validation.tsv', sep='\t').drop('Argument ID', axis=1)

print(labels_training.shape)
labels_training.head()


#new added label

label_to_add = ["Stance"]

# fixing the dataset for inconsistency: in favor of, in favour of
arguments_training_no_label['Stance'].replace(['in favour of'],
                       ['in favor of'], inplace=True)

arguments_validation_no_label['Stance'].replace(['in favour of'],
                       ['in favor of'], inplace=True)


arguments_test_data['Stance'].replace(['in favour of'],
                       ['in favor of'], inplace=True)

arguments_test_data_nahjalbalagha['Stance'].replace(['in favour of'],
                       ['in favor of'], inplace=True)

# from categorical value to numerical value for Stance

arguments_training_no_label['Stance'].replace(['in favor of', 'against'],
                       [1, 0], inplace=True)

arguments_validation_no_label['Stance'].replace(['in favor of', 'against'],
                       [1, 0], inplace=True)


arguments_test_data['Stance'].replace(['in favor of', 'against'],
                       [1, 0], inplace=True)

arguments_test_data_nahjalbalagha['Stance'].replace(['in favor of','against'],
                                                    [1,0], inplace=True)



(5393, 20)


In [ ]:
labels_training['Stance_As_Label'] = arguments_training_no_label[label_to_add] 
print(labels_training.shape)
print(labels_training.head())



labels_validation['Stance_As_Label'] = arguments_validation_no_label[label_to_add]
print(labels_validation.shape)
print(labels_validation.head())


In [ ]:
arguments_test_data_nahjalbalagha.head()

In [ ]:
# training set
train_labels = labels_training.values.tolist()

cols_to_concat = ["Conclusion", "Premise"]

arguments_training_no_label['Concated'] = arguments_training_no_label[cols_to_concat].apply(lambda x: '</s>'.join(x.astype(str)),
    axis=1)

print(arguments_training_no_label['Concated'][0])

#lowercase every word - roberta is case sensitive, thats why
arguments_training_no_label['Concated']= arguments_training_no_label['Concated'].str.lower()
print(arguments_training_no_label['Concated'][0])

train_text = arguments_training_no_label['Concated'].values
train_data = [[t, l] for t, l in zip(train_text, train_labels)] 
train_df = pd.DataFrame(train_data)
train_df.columns = ["text", "labels"]


We should ban human cloning</s>we should ban human cloning as it will only cause huge issues when you have a bunch of the same humans running around all acting the same.
we should ban human cloning</s>we should ban human cloning as it will only cause huge issues when you have a bunch of the same humans running around all acting the same.


In [ ]:
# tokenizer = AutoTokenizer.from_pretrained("roberta-base")
# tokenized = tokenizer.tokenize(train_df["text"][0])

In [ ]:
# tokenized

In [ ]:
# encoded = tokenizer.encode(train_df["text"][0], add_special_tokens=True)

In [ ]:
# encoded

In [ ]:
# ids_to_tokens= tokenizer.convert_ids_to_tokens(encoded)

In [ ]:
# ids_to_tokens

In [ ]:
# train_text[0]


In [ ]:
# print(train_df.head())
# print(train_labels[0:2])

In [ ]:

# repeat above for the validation set
val_labels = labels_validation.values.tolist()

arguments_validation_no_label['Concated'] = arguments_validation_no_label[cols_to_concat].apply(lambda x: '</s>'.join(x.astype(str)),
    axis=1)
print(arguments_validation_no_label['Concated'][0])

arguments_validation_no_label['Concated']= arguments_validation_no_label['Concated'].str.lower()
print(arguments_validation_no_label['Concated'][0])


val_text = arguments_validation_no_label['Concated'].values
val_data = [[t, l] for t, l in zip(val_text, val_labels)] 
val_df = pd.DataFrame(val_data)
val_df.columns = ["text", "labels"]




Entrapment should be legalized</s>if entrapment can serve to more easily capture wanted criminals, then why shouldn't it be legal?
entrapment should be legalized</s>if entrapment can serve to more easily capture wanted criminals, then why shouldn't it be legal?


In [ ]:
print(val_df.head())
print(val_df.shape)
print(len(val_labels[0]))

In [ ]:
# repeat above for the main test set

arguments_test_data['Concated'] = arguments_test_data[cols_to_concat].apply(lambda x: '</s>'.join(x.astype(str)),
    axis=1)

test_data = arguments_test_data['Concated'].values
print(np.shape(test_data))

print(arguments_test_data['Concated'][0])
arguments_test_data['Concated']= arguments_test_data['Concated'].str.lower()
print(arguments_test_data['Concated'][0])

(1576,)
We should end affirmative action</s>affirmative action helps with employment equity.
we should end affirmative action</s>affirmative action helps with employment equity.


In [ ]:
# repeat above for the nahjalbalagha test set

arguments_test_data_nahjalbalagha['Concated'] = arguments_test_data_nahjalbalagha[cols_to_concat].apply(lambda x: '</s>'.join(x.astype(str)),
    axis=1)

test_data_nahjalbalagha = arguments_test_data_nahjalbalagha['Concated'].values
print(np.shape(test_data_nahjalbalagha))
print(test_data_nahjalbalagha[0])

arguments_test_data_nahjalbalagha['Concated']= arguments_test_data_nahjalbalagha['Concated'].str.lower()
print(arguments_test_data_nahjalbalagha['Concated'][0])

(279,)
Habitual greed devalues humans</s>Greed is like a slippery rock on which even the steps of scholars are not firm.
habitual greed devalues humans</s>greed is like a slippery rock on which even the steps of scholars are not firm.


In [ ]:
for i in range(len(val_df)):
  for j in range(len(val_df['labels'][0])):
    if type(val_df['labels'][i][j]) == str:
      print(i,j, val_df['labels'][i][j] == 'in favour of')

In [ ]:
# Basic One
model_args = MultiLabelClassificationArgs(num_train_epochs=5, overwrite_output_dir=True)

# Create a MultiLabelClassificationModel from simple transformers: <https://simpletransformers.ai/docs/multi-label-classification/>
model = MultiLabelClassificationModel(
    "roberta",
    "roberta-base",
    num_labels=len(labels_training.columns),
    args=model_args,
)

# Train the model
model.train_model(train_df)

In [ ]:
#import sklearn


In [ ]:
# model_args = MultiLabelClassificationArgs()
# model_args.use_early_stopping = True
# model_args.early_stopping_delta = 0.01
# model_args.early_stopping_metric = "mcc"
# model_args.early_stopping_metric_minimize = False
# model_args.early_stopping_patience = 5
# model_args.evaluate_during_training_steps = 1000

# model = MultiLabelClassificationModel(
#     "roberta", 
#     "roberta-base", args=model_args)

# model.train_model(train_df)

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(val_df) #model.eval_model(eval_df, acc=sklearn.metrics.accuracy_score)

  0%|          | 0/1896 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/237 [00:00<?, ?it/s]

In [ ]:
result

{'LRAP': 0.7373263298671086, 'eval_loss': 0.33865249465034986}

In [ ]:
# predictions for the main test set
test_preds = model.predict(list(test_data))

  0%|          | 0/1576 [00:00<?, ?it/s]

  0%|          | 0/197 [00:00<?, ?it/s]

In [ ]:
# prediction outputs for the main test set
cols = ["Argument ID"] + list(labels_validation.columns)
pred_data = []

for arg_id, labels in zip(arguments_test_data["Argument ID"].values, test_preds[0]):
  row = [arg_id] + labels[:]
  pred_data.append(row)

pred_df = pd.DataFrame(pred_data, columns=cols)
pred_df_no_stance = pred_df.drop("Stance_As_Label",axis=1)

pred_df_no_stance.to_csv("main_test_pred.tsv", index=False, sep='\t')

In [ ]:
# predictions for the nahjalbalagha test set
test_preds_nahjalbalagha= model.predict(list(test_data_nahjalbalagha))

  0%|          | 0/279 [00:00<?, ?it/s]

  0%|          | 0/35 [00:00<?, ?it/s]

In [ ]:
# prediction outputs for the nahjalbalagha test set
cols = ["Argument ID"] + list(labels_validation.columns)
pred_data_nah = []

for arg_id, labels in zip(arguments_test_data_nahjalbalagha["Argument ID"].values, test_preds_nahjalbalagha[0]):
  row = [arg_id] + labels[:]
  pred_data_nah.append(row)

pred_nah_df = pd.DataFrame(pred_data_nah, columns=cols)
pred_df_nah_no_stance = pred_nah_df.drop("Stance_As_Label",axis=1)

pred_df_nah_no_stance.to_csv("nahjalbalagha_test_pred.tsv", index=False, sep='\t')

In [ ]:
pred_df_nah_no_stance.shape

(279, 21)

In [ ]:
pred_df_no_stance.shape

(1576, 21)

In [ ]:
! cp /content/outputs/pytorch_model.bin /content/drive/MyDrive/pytorch_model.bin 